In [188]:
import xlwings as xw
import pandas as pd
import timestring
import numpy as np

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.utils import shuffle
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import matplotlib.ticker as ticker

from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_absolute_error

from sklearn.metrics import mean_squared_error

from sklearn.model_selection import KFold

import matplotlib.pyplot as plt

### Загрузка и обработка данных по ценам и позициям

In [189]:
# загружаем данные по ценам и объему
data_rts = pd.read_csv('C:/Users/nicol/Python/Рынок/Эксперимент ФР/Эксперименты excel/SPFB.Si.txt', sep=',', index_col=2)

# меняем формат столбца и делаем в качестве индекса
data_rts.index = pd.to_datetime(data_rts.index, format="%Y%m%d")

# создадим столбец со средней ценой в минуту
data_rts['price_mean'] = data_rts['<CLOSE>']

# отсортируем по дате и возьмем среденее по цене за день
data_rts_mean = data_rts.pivot_table(index=['<DATE>'], values = 'price_mean', aggfunc=('mean'))

# открываем и читаем книгу по данным ФЗ ЮЛ
wb = xw.Book('C:/Users/nicol/Python/Рынок/Эксперимент ФР/Эксперименты excel/Юр-Физ лица цена2.xlsx') # Открываем книгу
data_excel = wb.sheets['Si-12.20'] # Читаем лист Данные

# считываем диапазон
data_pd = data_excel.range('A1:F867').options(pd.DataFrame, index = False).value

# меняем формат столбца 
data_pd['Дата'] = pd.to_datetime(data_pd['Дата'])

# устанавливаем в качестве индекса дату и удаляем лишний столбцец
data_pd.index = data_pd['Дата']
data_pd = data_pd.drop(['Дата'], axis = 1)


# загружаем данные по ценам и объему
data_rts_finam = pd.read_csv('C:/Users/nicol/Python/Рынок/Эксперимент ФР/Эксперименты excel/SPFB.Si-9.21.csv', sep=',', index_col=2)

# меняем формат столбца и делаем в качестве индекса
data_rts_finam.index = pd.to_datetime(data_rts_finam.index, format="%Y%m%d")

data_rts_finam['price_mean'] = data_rts_finam['<CLOSE>'] 

data_rts_finam_2 = data_rts_finam.drop(['<TICKER>','<PER>','<TIME>','<OPEN>','<HIGH>','<LOW>','<CLOSE>','<VOL>'], axis = 1)

data_rts_sum = pd.concat([data_rts_mean,data_rts_finam_2], axis = 0)

# выбираем из первой таблицы актив и соединяем таблицы с ценами
data_pd_n = data_pd.join(data_rts_sum, how = 'left')

# удаляем пробелы в цифрах
data_pd_n = data_pd_n.replace(r'\s+','',regex=True)

# переводим в числовой тип
data_pd_n = data_pd_n.apply(pd.to_numeric, errors='coerce')

# удаляем столбцы с низким значением позиций юрлиц
data_pd_n = data_pd_n.loc[data_pd_n['ЮЛДП'] > 1000]

data_pd_rts = data_pd_n

# добавим в % позиционирование ФЗ ЮЛ
data_pd_rts['ФЗДП%'] = data_pd_rts['ФЗДП']/(data_pd_rts['ФЗДП'] + data_pd_rts['ФЗКП'])
data_pd_rts['ЮЛКП%'] = data_pd_rts['ЮЛКП']/(data_pd_rts['ЮЛДП'] + data_pd_rts['ЮЛКП'])
data_pd_rts['ЮЛДП%'] = data_pd_rts['ЮЛДП']/(data_pd_rts['ЮЛДП'] + data_pd_rts['ЮЛКП'])
data_pd_rts['ФЗ_ЮЛ'] =(data_pd_rts['ФЗДП'] + data_pd_rts['ФЗКП'])/(data_pd_rts['ЮЛДП'] + data_pd_rts['ЮЛКП'])

data_pd_rts.sort_index(inplace=True, ascending=True)

In [190]:
#data_pd_rts.tail()

#### Даты эксперации квартального фьючерса 

In [191]:
data_experac = wb.sheets['Дата экспер'] # Читаем лист Данные

# считываем диапазон
experac = data_experac.range('C5:C17').options(pd.Series, index = False).value

# меняем формат столбца 
experac = pd.to_datetime(experac)

#### Даты эксперации месячных опционов

In [192]:
data_experac_opc = wb.sheets['Дата экспер'] # Читаем лист Данные

# считываем диапазон
experac_opc = data_experac_opc.range('E5:E41').options(pd.Series, index = False).value

# меняем формат столбца 
experac_opc = pd.to_datetime(experac_opc)

#### Создание графика ФЗ_ЮЛ

In [193]:
def train(window1, window2, window3):  
    
    '''
    Фичи: 
    
    1. window1 - скользящее среднее для цены
    2. window2 - скользящее среднее для ЮЛДП%_ФЗДП% 
    3. window3 - скользящее среднее для ФЗ_ЮЛ
    
    '''
    train = pd.DataFrame()

    # добавим СКСР для цены window1
   
    sksr_price = data_pd_rts['price_mean'].shift().rolling(window=window1).median()
    
    # добавим СКСР для ЮЛДП%_ФЗДП% window2
    sksr_ЮЛДП_ФЗДП = pd.DataFrame()
    sksr_ЮЛДП_ФЗДП['ewmaФЗДП%'] = data_pd_rts['ФЗДП%'].shift().rolling(window=window2).median()
    sksr_ЮЛДП_ФЗДП['ewmaЮЛДП%'] = data_pd_rts['ЮЛДП%'].shift().rolling(window=window2).median()
    sksr_ЮЛДП_ФЗДП['ЮЛДП%_ФЗДП%_sksr'] = sksr_ЮЛДП_ФЗДП['ewmaЮЛДП%']/sksr_ЮЛДП_ФЗДП['ewmaФЗДП%']
    

    # ЮЛДП%_ФЗДП% обычная не СКСР
    
    train['ЮЛДП%_ФЗДП%'] = data_pd_rts['ЮЛДП%']/data_pd_rts['ФЗДП%']
    
    
   
    # добавим СКСР объема позиций ФЗ к ЮЛ window3
    
    sksr_ФЗ_ЮЛ = data_pd_rts['ФЗ_ЮЛ'].shift().rolling(window=window3).median()
    
    # вставим все столбцы в train
    
        
    train['price_sksr'] = sksr_price
   

    train['ЮЛДП%_ФЗДП%_sksr'] = train['ЮЛДП%_ФЗДП%'].resample('3D').mean()
    train['ЮЛДП%_ФЗДП%_sksr'] = train['ЮЛДП%_ФЗДП%_sksr'].fillna(method = 'ffill')
   

    train['ФЗ_ЮЛ_sksr'] = sksr_ФЗ_ЮЛ
    
    
    train['ewmaФЗДП%'] = sksr_ЮЛДП_ФЗДП['ewmaФЗДП%']
    train['ewmaЮЛДП%'] = sksr_ЮЛДП_ФЗДП['ewmaЮЛДП%']
    
    # ФЗДП и ЮЛДП без скср
    train['ФЗДП%'] = data_pd_rts['ФЗДП%']
    train['ЮЛДП%'] = data_pd_rts['ЮЛДП%']
 
    train['price_mean'] = data_pd_rts['price_mean']
    
    #train = train.dropna().reset_index()
    #train = train.drop(['index'], axis =1)

    return train

In [194]:
# данные для построения графика ФЗ_ЮЛ
z = 2
opit_1 = train(z,z,z)

In [195]:
#opit_1.head(10)

### Создание осциллятора

In [196]:
def data_new(data_pd_rts, window1, window2, window3, window4):  
    
    '''
    Фичи (window1, window2, window3, window4): 
    
    1. window1 - СКСР price_mean
    2. window3 - СКСР объема позиций ФЗ к ЮЛ
    3. window2 - СКСР ЮЛДП%_ФЗДП% 
    4. window4 - СКСР ЮЛДП%_ФЗДП%_otnosit
           
    Цель:
    
    '''
    train = pd.DataFrame()

    # добавим СКСР для цены window1

    sksr_price = data_pd_rts['price_mean'].shift().rolling(window=window1).median()
    
    
    train['price_sksr'] = sksr_price

    train['price_mean'] = data_pd_rts['price_mean']
    

   
    # добавим СКСР для ЮЛДП%_ФЗДП% window2
    sksr_ЮЛДП_ФЗДП = pd.DataFrame()
    sksr_ЮЛДП_ФЗДП['ewmaФЗДП%'] = data_pd_rts['ФЗДП%'].shift().rolling(window=window2).median()
    sksr_ЮЛДП_ФЗДП['ewmaЮЛДП%'] = data_pd_rts['ЮЛДП%'].shift().rolling(window=window2).median()
    
    sksr_ЮЛДП_ФЗДП['ЮЛДП%_ФЗДП%_sksr'] = sksr_ЮЛДП_ФЗДП['ewmaЮЛДП%']/sksr_ЮЛДП_ФЗДП['ewmaФЗДП%']
    
    # отношение ЮЛДП%_ФЗДП% без скср
    sksr_ЮЛДП_ФЗДП['ЮЛДП%_ФЗДП%'] = data_pd_rts['ЮЛДП%']/data_pd_rts['ФЗДП%']
    
    
    # добавим СКСР объема позиций ФЗ к ЮЛ window3
    sksr_ФЗ_ЮЛ = data_pd_rts['ФЗ_ЮЛ'].shift().rolling(window=window3).median()
   
    
    train['ЮЛДП%_ФЗДП%_sksr'] = sksr_ЮЛДП_ФЗДП['ЮЛДП%_ФЗДП%_sksr']
    
    train['ЮЛДП%_ФЗДП%'] = sksr_ЮЛДП_ФЗДП['ЮЛДП%_ФЗДП%']
    
    # отношение ЮЛДП%_ФЗДП% к ЮЛДП%_ФЗДП% скср
    
    train['ЮЛДП%_ФЗДП%_otnosit'] = (train['ЮЛДП%_ФЗДП%']-train['ЮЛДП%_ФЗДП%_sksr'])/train['ЮЛДП%_ФЗДП%_sksr']
    
    # скользящее среднее для относительного значения
    
    train['ЮЛДП%_ФЗДП%_otnosit_sksr'] = train['ЮЛДП%_ФЗДП%_otnosit'].shift().rolling(window=window4).median()
        
    
    train['ФЗ_ЮЛ'] = data_pd_rts['ФЗ_ЮЛ']

    train['ФЗ_ЮЛ_sksr'] = sksr_ФЗ_ЮЛ
    
    # отношение ФЗ_ЮЛ к ФЗ_ЮЛ скср
    train['ФЗ_ЮЛ_otnosit'] = (train['ФЗ_ЮЛ']-train['ФЗ_ЮЛ_sksr'])/train['ФЗ_ЮЛ_sksr']
    
    
    train['ewmaФЗДП%'] = sksr_ЮЛДП_ФЗДП['ewmaФЗДП%']
    train['ewmaЮЛДП%'] = sksr_ЮЛДП_ФЗДП['ewmaЮЛДП%']

    return train

#### Экспорт осциллятора в отдельный файл для RTS

In [197]:
opit = data_new(data_pd_rts, 61, 51, 6, 13) 
opit.to_csv('opit_si.csv')

fzul_si = opit_1
fzul_si.to_csv('fzul_si.csv')

In [198]:
#fzul_si

In [199]:
#opit_1

### График ФЗ_ЮЛ

In [200]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
go.Scatter(x=data_pd_rts.index, y=opit_1['price_mean'], 
            name = 'price_mean',  line=dict(color='red', width=3)),
            secondary_y=True)

fig.add_trace(
go.Scatter(x=data_pd_rts.index, y=opit_1['price_sksr'], 
            name = 'price_sksr',  line=dict(color='pink', width=2)),
            secondary_y=True)

fig.add_trace(
go.Scatter(x=data_pd_rts.index, y=opit_1['ФЗДП%'], 
            name = 'ewmaФЗДП%',  line=dict(color='green', width=2)),
            secondary_y=False)

fig.add_trace(
go.Scatter(x=data_pd_rts.index, y=opit_1['ЮЛДП%'], 
            name = 'ewmaЮЛДП%',  line=dict(color='brown', width=2)),
            secondary_y=False)

fig.add_trace(
go.Scatter(x=data_pd_rts.index, y=opit_1['ФЗ_ЮЛ_sksr'],
            name = 'ФЗ_ЮЛ_sksr', line=dict(color='yellow', width=3)),
            secondary_y=False)

fig.update_yaxes(title_text="отн.знач.", tick0 = 0.1, dtick = 0.1, secondary_y=False)
fig.update_yaxes(title_text="Цена", tick0 = 2000, dtick = 1000, range = (60000,83000), secondary_y=True)

fig.update_layout(xaxis_title="Дата",height = 600, title="ФЗ и ЮЛ отдельно",
legend=dict(orientation="h", yanchor="bottom", 
y=1.02, xanchor="right", x=1 ))
       
fig.show()


fig2 = make_subplots(specs=[[{"secondary_y": True}]])

fig2.add_trace(
go.Scatter(x=data_pd_rts.index, y=opit_1['price_mean'], 
            name = 'price_mean',  line=dict(color='red', width=3)),
            secondary_y=True)

fig2.add_trace(
go.Scatter(x=data_pd_rts.index, y=opit_1['price_sksr'], 
            name = 'price_sksr',  line=dict(color='pink', width=3)),
            secondary_y=True)

fig2.add_trace(
go.Scatter(x=data_pd_rts.index, y=opit_1['ЮЛДП%_ФЗДП%_sksr'],
            name = 'ЮЛДП%_ФЗДП%_sksr', line=dict(color='purple', width=3)),
            secondary_y=False)

fig2.add_trace(
go.Scatter(x=data_pd_rts.index, y=opit_1['ЮЛДП%_ФЗДП%'],
            name = 'ЮЛДП%_ФЗДП%', line=dict(color='lightsteelblue', width=2)),
            secondary_y=False)

fig2.add_trace(
go.Scatter(x=data_pd_rts.index, y=opit_1['ФЗ_ЮЛ_sksr'],
            name = 'ФЗ_ЮЛ_sksr', line=dict(color='yellow', width=3)),
            secondary_y=False)


#fig.update_yaxes(title_text="отн.знач.", tick0 = 0.1, dtick = 0.1, secondary_y=False)
fig2.update_yaxes(title_text="Цена", secondary_y=True)
fig2.update_yaxes(tick0 = 2000, dtick = 1000, range = (60000,83000), secondary_y=True)

fig2.update_layout(xaxis_tickformat = "%d %B (%a)<br>%Y",title="ФЗ_ЮЛ",xaxis_title="Дата",height = 600,
legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1 ))
       
fig2.show()

### График осциллятора

In [201]:
fig3 = make_subplots(specs=[[{"secondary_y": True}]])

fig3.add_trace(
go.Scatter(x=data_pd_rts.index, y=data_pd_rts['price_mean'], 
            name = 'price_mean',  line=dict(color='red', width=3)),
            secondary_y=True)

# fig3.add_trace(
# go.Scatter(x=data_pd_rts.index, y=opit['price_sksr'], 
#             name = 'price_sksr',  line=dict(color='pink', width=3)),
#             secondary_y=True)

fig3.add_trace(
go.Scatter(x=data_pd_rts.index, y=opit['ЮЛДП%_ФЗДП%_otnosit_sksr'],
            name = 'ЮЛДП%_ФЗДП%_otnosit_sksr', line=dict(color='purple', width=2)),
            secondary_y=False)

fig3.add_trace(
go.Scatter(x=data_pd_rts.index, y=np.zeros((len(data_pd_rts.index))),
            line=dict(color='green', width=1)),
            secondary_y=False)

# даты экспирации месячных опционов
for i in range(len(experac_opc)):
    oy = np.arange(60,85)
    ox = np.full((len(oy)),experac_opc[i])
    oxs = pd.Series(ox) 
    fig3.add_trace(
    go.Scatter(x=oxs, y=oy*1000,
                line=dict(color='lightblue', width=1)),
                secondary_y=True)

# даты экспирации фьючерсов
for i in range(len(experac)):
    oy = np.arange(60,85)
    ox = np.full((len(oy)),experac[i])
    oxs = pd.Series(ox) 
    fig3.add_trace(
    go.Scatter(x=oxs, y=oy*1000,
                line=dict(color='blue', width=1)),
                secondary_y=True)

fig3.update_yaxes(title_text="Цена",range = (60000,83000), secondary_y=True)
fig3.update_yaxes(tick0 = 5000, dtick = 1000, secondary_y=True)

fig3.update_layout(
 xaxis_tickformat = "%d %B (%a)<br>%Y",xaxis_title="Дата",title="Осциллятор ФЗ_ЮЛ",height = 600)

fig3.update(layout_showlegend=False)  
       
fig3.show()

#### Некоторые дополнительные эксперименты

Условие что цена больше скользящей средней

In [202]:
# opit['price_bolshe_sksr'] = opit['price_mean'] > opit['price_sksr']

# opit=opit.applymap(lambda x: int(x) if type(x)==bool else x)
# opit = opit.dropna()
# opit[(opit['ЮЛДП%_ФЗДП%_otnosit_sksr'] > 0) & (opit['ФЗ_ЮЛ_otnosit'] > 0)]['price_bolshe_sksr'].hist()

In [203]:
# c = opit[(opit['ЮЛДП%_ФЗДП%_otnosit_sksr'] > 0) & (opit['ФЗ_ЮЛ_otnosit'] > 0)]['price_bolshe_sksr'].sum()
# d = opit[(opit['ЮЛДП%_ФЗДП%_otnosit_sksr'] > 0) & (opit['ФЗ_ЮЛ_otnosit'] > 0)]['price_bolshe_sksr'].count()
# c/d

In [204]:
# opit[(opit['ЮЛДП%_ФЗДП%_otnosit_sksr'] > 0 ) & (opit['ФЗ_ЮЛ_otnosit'] > 0.5)]['price_bolshe_sksr'].hist()

Подбор параметров windows 1, 2, 3 для более достоверных условий что цена больше скср

In [205]:
# mod_1 = pd.DataFrame()
# n = 0
# for window1 in range(1, 207, 5):
#     for window2 in range(1, 207, 5):
#         for window3 in range(1, 207, 5):
#             for window4 in range(5, 16, 1):
            
#                 n += 1
#                 opit = data_new(data_pd_rts, window1, window2, window3, window4)
#                 opit['price_bolshe_sksr'] = opit['price_mean'] > opit['price_sksr']

#                 opit=opit.applymap(lambda x: int(x) if type(x)==bool else x)
#                 opit = opit.dropna()

#                 c = opit[(opit['ЮЛДП%_ФЗДП%_otnosit_sksr'] > 0) & (opit['ФЗ_ЮЛ_otnosit'] > 0)]['price_bolshe_sksr'].sum()
#                 d = opit[(opit['ЮЛДП%_ФЗДП%_otnosit_sksr'] > 0) & (opit['ФЗ_ЮЛ_otnosit'] > 0)]['price_bolshe_sksr'].count()

#                 count = c/d

#                 mod_1.loc[n, 'window1'] = window1
#                 mod_1.loc[n, 'window2'] = window2
#                 mod_1.loc[n, 'window3'] = window3
#                 mod_1.loc[n, 'window4'] = window4
#                 mod_1.loc[n, 'count'] = count

In [206]:
# mod_1.loc[mod_1['count'] == mod_1['count'].max()]

In [207]:
#mod_1.to_csv('осциллятор Si-6.21 13.04 шаг 5.1.csv')

In [208]:
#mod_1.loc[mod_1['count'] == mod_1['count'].max()]

In [209]:
# w = 61
# mod_1.loc[(mod_1['window1'] == w)].loc[mod_1.loc[(mod_1['window1'] == w)]['count'] == mod_1.loc[(mod_1['window1'] == w)]['count'].max()]

In [210]:
#mod_1.loc[mod_1['count'] == mod_1['count'].min()]

In [211]:
#100**3

In [212]:
# import requests

# import apimoex
# import pandas as pd


# with requests.Session() as session:
#     data = apimoex.get_market_candles(
#         session=session,
#         security="RIH1",
#         market="forts",
#         engine="futures",
#         interval=24,
#         start = "01.02.2021"
#     )
#     df = pd.DataFrame(data)
#     df.set_index("begin", inplace=True)
#     print(df.head(), "\n")
#     print(df.tail(), "\n")
#     df.info()

In [213]:
# import requests
# import datetime
# import pathlib
# import apimoex
# import pandas as pd

# board = 'TQBR'

# #with open("C:/PYEX/TICK.txt", "r") as TICKs:
#     TICKs = [line.rstrip() for line in TICKs]
# pathlib.Path("C:/PYEX/Database/{}".format(board)).mkdir(parents=True, exist_ok=True)
# process = 0
# with requests.Session() as session:
#     for TICK in TICKs:
#          process = process + 1
#          print((process / len(TICKs)) * 100, ' %')
#          data = apimoex.get_board_history(session, TICK, board=board)
#          if data == []:
#              continue
#          df = pd.DataFrame(data)
#          df = df[['TRADEDATE','CLOSE']]
#          df.to_excel("C:/PYEX/Database/{}/{}.xlsx".format(board,TICK), index=False)

In [214]:
# import requests
# import datetime
# import pathlib
# import apimoex
# import pandas as pd
# data = apimoex.get_board_history('GAZP', board='TQBR')

In [215]:
#pip install apimoex

In [216]:
# import requests
# import datetime
# import pathlib

# SECIDs = ["GAZP", "BANEP", "LKOH"]
# DISK = "H"
# for SECID in SECIDs:
#     from_date = "2020-05-04"
#     to_date = "2005-01-03"
#     while str(to_date) != from_date:
#         to_date = str(to_date)
#         to_date = to_date.split('-')
#         a = datetime.date(int(to_date[0]), int(to_date[1]), int(to_date[2]))
#         b = datetime.timedelta(days=140)
#         to_date = a + b
#         pathlib.Path("{}:/{}/{}".format(DISK, "Database_MOEX", SECID)).mkdir(parents=True, exist_ok=True)
#         filename = SECID + "_" + str(to_date) + ".csv"
#         with requests.get("http://iss.moex.com/iss/history/engines/stock/markets/shares/boards/tqbr/securities/{}.csv?date={}".format(SECID, to_date)) as response:
#             with open("{}:/Database_MOEX/{}/{}".format(DISK, SECID, filename), 'wb') as f:
#                 for chunk in response.iter_content():
#                     f.write(chunk)

### Import from quandl

In [217]:
# %matplotlib inline
# import os
# import quandl

# import seaborn as sns
# import matplotlib.pyplot as plt

In [218]:
# quandl.ApiConfig.api_key = "Apnzsot_QRPgvhw6WyMb"
# sns.set_style('whitegrid')
# oil = quandl.get('FRED/DCOILBRENTEU', start_date="2008-01-01")
# oil.plot(lw=2, title='EIA/PET_RWTC_D', figsize=(12, 4))
# sns.despine()
# plt.tight_layout();